# $\bm{19}$ $\enspace$ **卷积层**

## $\bm{19.1}$ $\enspace$ **从全连接层到卷积**

### $\bm{19.1.1}$ $\enspace$ **不变性**

$(1)$ $\enspace$ **平移不变性 (translation invariance)**：不管检测对象出现在图像中的哪个位置，神经网络的前面几层应该对相同的图像区域有相似的反应，即为“平移不变性”。

$(2)$ $\enspace$ **局部性 (locality)**：神经网络的前面几层应该只探索输入图像中的某些区域，而不过度在意图像中相隔较远区域的位置，这就是“局部性”原则。最终，可以聚合这些局部特征，以在整个图像级别进行预测。

### $\bm{19.1.2}$ $\enspace$ **多层感知机的限制**

首先，多层感知机的输入是二维图像 $\bm{X}$，其隐藏表示 $\bm{H}$ 在数学上是一个矩阵，在代码上表示为二维张量。其中，$\bm{X}$ 和 $\bm{H}$ 具有相同的形状。

使用 $[\bm{X}]_{i,j}$ 和 $[\bm{H}]_{i,j}$ 分别表示输入图像和隐藏表示中位置 $(i,j)$ 处的像素。为了使每个隐藏神经元都能接收到每个输入像素的信息，我们将参数从权重矩阵替换为四阶权重张量 $W$。假设 $\bm{U}$ 包含偏置参数，我们可以将全连接层形式化地表示为
$$
    \begin{split}
        [\bm{H}]_{i,j}&=[\bm{U}]_{i,j}+\sum_k \sum_l [W]_{i,j,k,l}[\bm{X}]_{k,l} \\
        &=[\bm{U}]_{i,j}+\sum_a \sum_b [V]_{i,j,a,b}[\bm{X}]_{i+a,j+b} \tag{1}
    \end{split}
$$
其中，从 $W$ 到 $V$ 的转换只是形式上的转换，因为在这两个四阶张量的元素之间存在一一对应的关系。我们只需重新索引下标 $(k,l)$，使 $k=i+a,\ l=j+b$，由此可得 $[V]_{i,j,a,b}=[W]_{i,j,k,l}$。

**平移不变性**

检测对象在输入 $\bm{X}$ 中的平移，应该仅导致隐藏表示 $\bm{H}$ 中的平移。也就是说，$V$ 和 $\bm{U}$ 实际上不依赖于 $(i,j)$ 的值，即 $[V]_{i,j,a,b}=[\bm{V}]_{a,b}$。并且 $\bm{U}$ 是一个常数，比如 $u$。因此，我们可以简化 $\bm{H}$ 定义为：
$$
    [\bm{H}]_{i,j}=u+\sum_a \sum_b [\bm{V}]_{a,b}[\bm{X}]_{i+a,j+b} \tag{2}
$$

**局部性**

为了收集用来训练参数 $[\bm{H}]_{i,j}$ 的相关信息，我们不应偏离到距 $(i,j)$ 很远的地方。这意味着在 $\vert a\vert >\Delta$ 或 $\vert b\vert >\Delta$ 的范围之外，我们可以设置 $[\bm{V}]_{a,b}=0$。因此，我们可以将 $[\bm{H}]_{i,j}$ 重写为
$$
    [\bm{H}]_{i,j}=u+\sum_{a=-\Delta}^\Delta \sum_{b=-\Delta}^\Delta [\bm{V}]_{a,b}[\bm{X}]_{i+a,j+b} \tag{3}
$$

### $\bm{19.1.3}$ $\enspace$ **卷积**

在数学中，两个函数（比如 $f,g: \mathbb{R}^d\rightarrow \mathbb{R}$）之间的“卷积”被定义为
$$
    (f*g)(\bm{x})=\int f(\bm{z})g(\bm{x}-\bm{z})\mathrm{d}\bm{z} \tag{4}
$$
也就是说，卷积是当把一个函数“翻转”并移位 $\bm{x}$ 时，测量 $f$ 和 $g$ 之间的重叠。当为离散对象时，积分就变成求和。例如，对于由索引为 $\mathbb{Z}$ 的、平方可和的、无限维向量集合中抽取的向量，我们可以得到以下定义：
$$
    (f*g)(i)=\sum_a f(a)g(i-a) \tag{5}
$$
对于二维张量，则为 $f$ 的索引 $(a,b)$ 和 $g$ 的索引 $(i-a,j-b)$ 上的对应加和：
$$
    (f*g)(i,j)=\sum_a \sum_b f(a,b)g(i-a,j-b) \tag{6}
$$

## $\bm{19.2}$ $\enspace$ **图像卷积**

### $\bm{19.2.1}$ $\enspace$ **互相关运算**

In [4]:
import torch
from torch import nn
from d2l import torch as d2l


def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i : i + h, j : j + w] * K).sum()
    return Y

验证二维互相关运算的输出

In [5]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

### $\bm{19.2.2}$ $\enspace$ **卷积层**

基于上面定义的 `corr2d` 函数实现二维卷积层。在 `__init__` 构造函数中，将 `weight` 和 `bias` 声明为两个模型参数。前向传播函数调用 `corr2d` 函数并添加偏置。

In [7]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

### $\bm{19.2.3}$ $\enspace$ **图像中目标的边缘检测**

In [6]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

接下来，我们构造一个高度为 $1$，宽度为 $2$ 的卷积核 $K$。当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零。

In [8]:
K = torch.tensor([[1.0, -1.0]])

现在，我们对参数 $X$（输入）和 $K$（卷积核）执行互相关运算。

In [9]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

现在我们将输入的二维图像转置，再进行如上的互相关运算。其输出如下，之前检测到的垂直边缘消失了。不出所料，这个卷积核 $K$ 只可以检测垂直边缘，无法检测水平边缘。

In [10]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

### $\bm{19.2.4}$ $\enspace$ **学习卷积核**

我们先构造⼀个卷积层，并将其卷积核初始化为随机张量。接下来，在每次迭代中，我们比较 $Y$ 与卷积层输出的平方误差，然后计算梯度来更新卷积核。为了简单起见，我们在此使用内置的二维卷积层，并忽略偏置。

In [11]:
# 构造一个二维卷积层，它具有1个输出通道和形状为 (1,2) 的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度）
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f"epoch {i+1}, loss {l.sum():.3f}")

epoch 2, loss 10.240
epoch 4, loss 1.796
epoch 6, loss 0.333
epoch 8, loss 0.069
epoch 10, loss 0.017


所学的卷积核的权重张量

In [12]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9738, -0.9935]])